In [14]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings('ignore')

In [15]:
# Step 1: Create dataset
X, y = make_classification(
    n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
    weights=[0.9, 0.1], flip_y=0, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


In [16]:
# Model params
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 42,
}


In [17]:
# Train
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Eval
y_pred = lr.predict(X_test)
report_dict = classification_report(y_test, y_pred, output_dict=True)
auc = roc_auc_score(y_test, lr.predict_proba(X_test)[:, 1])

# MLflow logging
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("1st Experiment")

with mlflow.start_run(run_name="logreg_v1"):
    mlflow.log_params(params)
    mlflow.log_metrics({
        "accuracy": report_dict["accuracy"],
        "recall_class_0": report_dict["0"]["recall"],
        "recall_class_1": report_dict["1"]["recall"],
        "f1_score_macro": report_dict["macro avg"]["f1-score"],
        "roc_auc": auc
    })
    input_example = pd.DataFrame(X_train[:1], columns=[f"feature_{i}" for i in range(X_train.shape[1])])
    mlflow.sklearn.log_model(
        sk_model=lr,
        name="Logistic Regression",
        input_example=input_example,
        registered_model_name="log_reg_model"
    )

Successfully registered model 'log_reg_model'.
2025/08/24 05:24:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: log_reg_model, version 1
Created version '1' of model 'log_reg_model'.


🏃 View run logreg_v1 at: http://127.0.0.1:5000/#/experiments/978891286603990841/runs/dc0c62c818ef49308c1d08c12d65a2ed
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/978891286603990841
